### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import plotly.plotly as py
import plotly.graph_objs as go
import math
%matplotlib inline

# my functions
import external.dm_helpers as hdm
import external.text_analysis as ta

In [2]:
# necessary for when working with external scripts
%load_ext autoreload
%autoreload 2

In [3]:
from sklearn.datasets import load_files
twenty_train = load_files('/Users/Vinit/Documents/_SEM6/Project/DM/dataMiningProject/20newsbydate/20news4cat')


In [4]:
print twenty_train.keys()


['target_names', 'data', 'target', 'DESCR', 'filenames']


In [5]:
print twenty_train.target_names 


['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


In [6]:
print twenty_train.data[0:2]

['From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul Checkman)\nSubject: Re: Is MSG sensitivity superstition?\nArticle-I.D.: news.C5wI4F.Dt\nOrganization: University of Illinois at Urbana\nLines: 22\n\nbruce@Data-IO.COM (Bruce Reynolds) writes:\n\n>Anecedotal evidence is worthless.  Even doctors who have been using a drug\n>or treatment for years, and who swear it is effective, are often suprised\n>at the results of clinical trials.  Whether or not MSG causes describable,\n>reportable, documentable symptoms should be pretty simple to discover.  \n\nI tend to disagree- I think anecdotal evidence, provided there is a lot of it,\nand it is fairly consistent, will is very important.  First, it points to the\nnecessity of doing a study, and second, it at least says that the effects are\nall psychological (or possibly allergy in this case).  As I\'ve pointed out \nbefore, pyschological effects are no less real than other effects.  One       \nperson\'s "make-believe" can easily be another person\'

### Clear vision

In [7]:
print("\n".join(twenty_train.data[0].split("\n")))

From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul Checkman)
Subject: Re: Is MSG sensitivity superstition?
Article-I.D.: news.C5wI4F.Dt
Organization: University of Illinois at Urbana
Lines: 22

bruce@Data-IO.COM (Bruce Reynolds) writes:

>Anecedotal evidence is worthless.  Even doctors who have been using a drug
>or treatment for years, and who swear it is effective, are often suprised
>at the results of clinical trials.  Whether or not MSG causes describable,
>reportable, documentable symptoms should be pretty simple to discover.  

I tend to disagree- I think anecdotal evidence, provided there is a lot of it,
and it is fairly consistent, will is very important.  First, it points to the
necessity of doing a study, and second, it at least says that the effects are
all psychological (or possibly allergy in this case).  As I've pointed out 
before, pyschological effects are no less real than other effects.  One       
person's "make-believe" can easily be another person's reality.  Using 
psyc

In [8]:
print twenty_train.filenames

[ '/Users/Vinit/Documents/_SEM6/Project/DM/dataMiningProject/20newsbydate/20news4cat/sci.med/59184'
 '/Users/Vinit/Documents/_SEM6/Project/DM/dataMiningProject/20newsbydate/20news4cat/comp.graphics/38735'
 '/Users/Vinit/Documents/_SEM6/Project/DM/dataMiningProject/20newsbydate/20news4cat/soc.religion.christian/20968'
 ...,
 '/Users/Vinit/Documents/_SEM6/Project/DM/dataMiningProject/20newsbydate/20news4cat/comp.graphics/38457'
 '/Users/Vinit/Documents/_SEM6/Project/DM/dataMiningProject/20newsbydate/20news4cat/comp.graphics/38530'
 '/Users/Vinit/Documents/_SEM6/Project/DM/dataMiningProject/20newsbydate/20news4cat/sci.med/59240']


In [9]:
#categories = ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


### Assigning which target names to which target

In [10]:
print twenty_train.target[:10]

[2 1 3 2 0 2 2 1 1 0]


In [11]:
temp, target_list = [],[]
for t in twenty_train.target[:50]:
    print(twenty_train.target_names[t])
    target_list.append(twenty_train.target_names[t]) 

print "Total number of different categories - ",set(target_list)

sci.med
comp.graphics
soc.religion.christian
sci.med
alt.atheism
sci.med
sci.med
comp.graphics
comp.graphics
alt.atheism
comp.graphics
comp.graphics
comp.graphics
sci.med
alt.atheism
soc.religion.christian
soc.religion.christian
sci.med
comp.graphics
alt.atheism
sci.med
soc.religion.christian
soc.religion.christian
alt.atheism
comp.graphics
soc.religion.christian
comp.graphics
comp.graphics
alt.atheism
alt.atheism
sci.med
sci.med
soc.religion.christian
sci.med
alt.atheism
alt.atheism
alt.atheism
comp.graphics
soc.religion.christian
soc.religion.christian
alt.atheism
comp.graphics
comp.graphics
soc.religion.christian
alt.atheism
soc.religion.christian
sci.med
comp.graphics
comp.graphics
alt.atheism
Total number of different categories -  set(['sci.med', 'soc.religion.christian', 'alt.atheism', 'comp.graphics'])


In [12]:
len(twenty_train.data)

2257

## Love Pandas = Dictionary -> CSV file 

In [13]:
X = pd.DataFrame.from_records(hdm.rows_format(twenty_train), columns= ['text'])


In [14]:
len(X)

2257

In [15]:
X[0:20]

,text
0,From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul C...
1,From: yoo@engr.ucf.edu (Hoi Yoo) Subject: look...
2,From: fernandeza@merrimack.edu Subject: Re: Th...
3,From: mcelwre@cnsvax.uwec.edu Subject: NATURAL...
4,From: mathew <mathew@mantis.co.uk> Subject: Re...
5,From: uabdpo.dpo.uab.edu!gila005 (Stephen Holl...
6,From: ron.roth@rose.com (ron roth) Subject: Se...
7,From: bates@spica.ucsb.edu (Andrew M. Bates) S...
8,From: billj@b11.b11.ingr.com (Bill Jones) Subj...
9,From: madhaus@netcom.com (Maddi Hausmann) Subj...


In [16]:
# add category to the dataframe
X['category'] = twenty_train.target

In [17]:
print twenty_train.target_names
print twenty_train.target[0:4]

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']
[2 1 3 2]


In [18]:
X['category_name'] = X.category.apply(lambda t: hdm.labels_format(t, twenty_train))


In [19]:
X[0:20]

,text,category,category_name
0,From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul C...,2,sci.med
1,From: yoo@engr.ucf.edu (Hoi Yoo) Subject: look...,1,comp.graphics
2,From: fernandeza@merrimack.edu Subject: Re: Th...,3,soc.religion.christian
3,From: mcelwre@cnsvax.uwec.edu Subject: NATURAL...,2,sci.med
4,From: mathew <mathew@mantis.co.uk> Subject: Re...,0,alt.atheism
5,From: uabdpo.dpo.uab.edu!gila005 (Stephen Holl...,2,sci.med
6,From: ron.roth@rose.com (ron roth) Subject: Se...,2,sci.med
7,From: bates@spica.ucsb.edu (Andrew M. Bates) S...,1,comp.graphics
8,From: billj@b11.b11.ingr.com (Bill Jones) Subj...,1,comp.graphics
9,From: madhaus@netcom.com (Maddi Hausmann) Subj...,0,alt.atheism


In [20]:
dataset = X
print dataset[:5]

                                                text  category  \
0  From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul C...         2   
1  From: yoo@engr.ucf.edu (Hoi Yoo) Subject: look...         1   
2  From: fernandeza@merrimack.edu Subject: Re: Th...         3   
3  From: mcelwre@cnsvax.uwec.edu Subject: NATURAL...         2   
4  From: mathew <mathew@mantis.co.uk> Subject: Re...         0   

            category_name  
0                 sci.med  
1           comp.graphics  
2  soc.religion.christian  
3                 sci.med  
4             alt.atheism  


### Pandas made easy to play with dataset

In [21]:
X[0:10][["text", "category_name"]]

,text,category_name
0,From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul C...,sci.med
1,From: yoo@engr.ucf.edu (Hoi Yoo) Subject: look...,comp.graphics
2,From: fernandeza@merrimack.edu Subject: Re: Th...,soc.religion.christian
3,From: mcelwre@cnsvax.uwec.edu Subject: NATURAL...,sci.med
4,From: mathew <mathew@mantis.co.uk> Subject: Re...,alt.atheism
5,From: uabdpo.dpo.uab.edu!gila005 (Stephen Holl...,sci.med
6,From: ron.roth@rose.com (ron roth) Subject: Se...,sci.med
7,From: bates@spica.ucsb.edu (Andrew M. Bates) S...,comp.graphics
8,From: billj@b11.b11.ingr.com (Bill Jones) Subj...,comp.graphics
9,From: madhaus@netcom.com (Maddi Hausmann) Subj...,alt.atheism


#### Last 10 records

In [22]:
X[-11:-1]

,text,category,category_name
2246,From: ejalbert@husc3.harvard.edu Subject: Re: ...,3,soc.religion.christian
2247,From: maridai@comm.mot.com (Marida Ignacio) Su...,3,soc.religion.christian
2248,From: Mike_Peredo@mindlink.bc.ca (Mike Peredo)...,1,comp.graphics
2249,From: cctr114@cantua.canterbury.ac.nz (Bill Re...,3,soc.religion.christian
2250,From: dlecoint@garnet.acns.fsu.edu (Darius_Lec...,3,soc.religion.christian
2251,From: keith@cco.caltech.edu (Keith Allan Schne...,0,alt.atheism
2252,From: ib@ivan.asd.sgi.com (Ivan Bach) Subject:...,1,comp.graphics
2253,From: topcat!tom@tredysvr.tredydev.unisys.com ...,3,soc.religion.christian
2254,From: sean@whiting.mcs.com (Sean Gum) Subject:...,1,comp.graphics
2255,From: dingebre@imp.sim.es.com (David Ingebrets...,1,comp.graphics


In [23]:
X.iloc[::10, :][0:10]
# print dataset[0:100:10]

,text,category,category_name
0,From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul C...,2,sci.med
10,From: davidr@rincon.ema.rockwell.com (David J....,1,comp.graphics
20,From: mryan@stsci.edu Subject: Should I be ang...,2,sci.med
30,From: jchen@wind.bellcore.com (Jason Chen) Sub...,2,sci.med
40,From: I3150101@dbstu1.rz.tu-bs.de (Benedikt Ro...,0,alt.atheism
50,From: mmm@cup.portal.com (Mark Robert Thorson)...,2,sci.med
60,From: jodfishe@silver.ucs.indiana.edu (joseph ...,3,soc.religion.christian
70,From: ab961@Freenet.carleton.ca (Robert Alliso...,2,sci.med
80,From: ak949@yfn.ysu.edu (Michael Holloway) Sub...,2,sci.med
90,From: keith@cco.caltech.edu (Keith Allan Schne...,0,alt.atheism


### Missing Values

In [24]:
X.isnull()

,text,category,category_name
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
6,False,False,False
7,False,False,False
8,False,False,False
9,False,False,False


In [25]:
X.isnull().apply(lambda x: hdm.missing_values_check(x))


text             (The amoung of missing records is: , 0)
category         (The amoung of missing records is: , 0)
category_name    (The amoung of missing records is: , 0)
dtype: object

In [26]:
dummy_dict = [{'text': 'dummy_record',
               'category': 1
              },
             {'text' : 'dummy_record2',
               'category_name' : 'alt.atheism' 
             }]

In [27]:
X = X.append(dummy_dict,ignore_index=True)

In [28]:
len(X)

2259

In [29]:
X.isnull()

,text,category,category_name
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
6,False,False,False
7,False,False,False
8,False,False,False
9,False,False,False


In [30]:
X.isnull().apply(lambda x: hdm.missing_values_check(x))


text             (The amoung of missing records is: , 0)
category         (The amoung of missing records is: , 1)
category_name    (The amoung of missing records is: , 1)
dtype: object

### To remove null values

In [31]:
X.dropna(inplace=True)
print len(X)

2257


In [32]:
X.isnull().apply(lambda x: hdm.missing_values_check(x))


text             (The amoung of missing records is: , 0)
category         (The amoung of missing records is: , 0)
category_name    (The amoung of missing records is: , 0)
dtype: object

### Handling Duplicate values

In [33]:
X.duplicated()

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
2227    False
2228    False
2229    False
2230    False
2231    False
2232    False
2233    False
2234    False
2235    False
2236    False
2237    False
2238    False
2239    False
2240    False
2241    False
2242    False
2243    False
2244    False
2245    False
2246    False
2247    False
2248    False
2249    False
2250    False
2251    False
2252    False
2253    False
2254    False
2255    False
2256    False
Length: 2257, dtype: bool

In [34]:
sum(X.duplicated())

0

In [35]:
dummy_duplicate_dict = [{
                             'text': 'dummy record',
                             'category': 1, 
                             'category_name': "dummy category"
                        },
                        {
                             'text': 'dummy record',
                             'category': 1, 
                             'category_name': "dummy category"
                        }]

In [36]:
X = X.append(dummy_duplicate_dict, ignore_index=True)

In [37]:
len(X)

2259

In [38]:
sum(X.duplicated())

1

In [39]:
X.drop_duplicates(keep=False, inplace=True)

In [40]:
len(X)

2257

In [41]:
sum(X.duplicated())

0

## Data Preprocessing
####  Sampling 
Reducing size of the dataset with minimum loss of information

In [42]:
X_sample = X.sample(n=1000)

In [43]:
len(X_sample)

1000

In [44]:
X_sample[0:4]

,text,category,category_name
930,From: bil@okcforum.osrhe.edu (Bill Conner) Sub...,0.0,alt.atheism
474,From: tom@inferno.UUCP (Tom Sherwin) Subject: ...,1.0,comp.graphics
472,From: a137490@lehtori.cc.tut.fi (Aario Sami) S...,0.0,alt.atheism
1514,From: johnsd2@rpi.edu (Dan Johnson) Subject: R...,3.0,soc.religion.christian


In [45]:
X_category_counts = ta.get_tokens_and_frequency(list(X.category_name))
X_sample_category_counts = ta.get_tokens_and_frequency(list(X_sample.category_name))

In [46]:
import plotly
plotly.tools.set_credentials_file(username='bodhwani', api_key='L5CXni0YCZ8FhGJ1LtqA')



In [47]:
py.iplot(ta.plot_word_frequency(X_category_counts, "Category distribution"))

In [48]:
py.iplot(ta.plot_word_frequency(X_sample_category_counts, "Category distribution"))

#### Feature Creation

In [49]:
# takes a like a minute or two to process
X['unigrams'] = X['text'].apply(lambda x: hdm.tokenize_text(x.decode('ISO-8859-1')))


In [50]:
X[0:4]["unigrams"]

0    [From, :, dpc47852, @, uxa.cso.uiuc.edu, (, Da...
1    [From, :, yoo, @, engr.ucf.edu, (, Hoi, Yoo, )...
2    [From, :, fernandeza, @, merrimack.edu, Subjec...
3    [From, :, mcelwre, @, cnsvax.uwec.edu, Subject...
Name: unigrams, dtype: object

In [51]:
list(X[0:1]['unigrams'])

[[u'From',
  u':',
  u'dpc47852',
  u'@',
  u'uxa.cso.uiuc.edu',
  u'(',
  u'Daniel',
  u'Paul',
  u'Checkman',
  u')',
  u'Subject',
  u':',
  u'Re',
  u':',
  u'Is',
  u'MSG',
  u'sensitivity',
  u'superstition',
  u'?',
  u'Article-I.D',
  u'.',
  u':',
  u'news.C5wI4F.Dt',
  u'Organization',
  u':',
  u'University',
  u'of',
  u'Illinois',
  u'at',
  u'Urbana',
  u'Lines',
  u':',
  u'22',
  u'bruce',
  u'@',
  u'Data-IO.COM',
  u'(',
  u'Bruce',
  u'Reynolds',
  u')',
  u'writes',
  u':',
  u'>',
  u'Anecedotal',
  u'evidence',
  u'is',
  u'worthless',
  u'.',
  u'Even',
  u'doctors',
  u'who',
  u'have',
  u'been',
  u'using',
  u'a',
  u'drug',
  u'>',
  u'or',
  u'treatment',
  u'for',
  u'years',
  u',',
  u'and',
  u'who',
  u'swear',
  u'it',
  u'is',
  u'effective',
  u',',
  u'are',
  u'often',
  u'suprised',
  u'>',
  u'at',
  u'the',
  u'results',
  u'of',
  u'clinical',
  u'trials',
  u'.',
  u'Whether',
  u'or',
  u'not',
  u'MSG',
  u'causes',
  u'describable',
  u','

#### Feature Selection

Example for understanding

In [52]:
temp_vec = CountVectorizer(encoding='ISO-8859-1')
string1 = "hello vinit"
string2 = "hello bodhwani"
email = [string1,string2]
temp = temp_vec.fit_transform(email)
print temp.toarray()
print temp_vec.get_feature_names()

[[0 1 1]
 [1 1 0]]
[u'bodhwani', u'hello', u'vinit']


In [53]:
count_vect = CountVectorizer(encoding='ISO-8859-1')
X_counts = count_vect.fit_transform(X['text'])


In [54]:
count_vect.get_feature_names()[-50:]

[u'zinc',
 u'zion',
 u'zionist',
 u'zip',
 u'zipped',
 u'zippy',
 u'zirkel',
 u'zisfein',
 u'zlumber',
 u'zmodem',
 u'zoerasterism',
 u'zola',
 u'zolf',
 u'zolft',
 u'zoloft',
 u'zombie',
 u'zonal',
 u'zone',
 u'zonker',
 u'zoo',
 u'zooid',
 u'zool',
 u'zoom',
 u'zooming',
 u'zooms',
 u'zopfi',
 u'zorasterism',
 u'zorg',
 u'zorn',
 u'zrz',
 u'zsoft',
 u'zt',
 u'zubkoff',
 u'zues',
 u'zug',
 u'zumder',
 u'zur',
 u'zurich',
 u'zurlo',
 u'zus',
 u'zvi',
 u'zvonko',
 u'zwart',
 u'zyeh',
 u'zyklon',
 u'zyxel',
 u'zz',
 u'zzz',
 u'\xaal',
 u'\xed\xe5littin']

In [55]:
count_vect.inverse_transform(X_counts[0])

[array([u'dan', u'done', u'being', u'leads', u'stuff', u'still',
        u'something', u'food', u'just', u'could', u'hand', u'affects',
        u'experience', u'definitely', u'he', u'then', u'bad', u'causing',
        u'perceives', u'ingesting', u'someone', u'long', u'perception',
        u'matter', u'point', u'my', u'made', u'hopefully', u'but', u'lame',
        u'was', u'anyhow', u'happen', u'going', u'always', u'those',
        u'guaranteed', u'cannot', u'trip', u'acid', u'an', u'on',
        u'experiences', u'hallucinations', u'example', u'twisted',
        u'bizarre', u'drugs', u'psychadelic', u'reality', u'another',
        u'easily', u'can', u'believe', u'make', u'person', u'one', u'other',
        u'than', u'real', u'less', u'no', u'pyschological', u'before',
        u'out', u'pointed', u've', u'as', u'case', u'this', u'in',
        u'allergy', u'possibly', u'psychological', u'all', u'effects',
        u'that', u'says', u'least', u'second', u'study', u'doing',
        u'necessi

In [56]:
X_counts[0:5,0:100].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0,

In [57]:
# We can check the shape of this matrix by:
X_counts.shape

(2257, 35788)

In [58]:
X[0:]

,text,category,category_name,unigrams
0,From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul C...,2.0,sci.med,"[From, :, dpc47852, @, uxa.cso.uiuc.edu, (, Da..."
1,From: yoo@engr.ucf.edu (Hoi Yoo) Subject: look...,1.0,comp.graphics,"[From, :, yoo, @, engr.ucf.edu, (, Hoi, Yoo, )..."
2,From: fernandeza@merrimack.edu Subject: Re: Th...,3.0,soc.religion.christian,"[From, :, fernandeza, @, merrimack.edu, Subjec..."
3,From: mcelwre@cnsvax.uwec.edu Subject: NATURAL...,2.0,sci.med,"[From, :, mcelwre, @, cnsvax.uwec.edu, Subject..."
4,From: mathew <mathew@mantis.co.uk> Subject: Re...,0.0,alt.atheism,"[From, :, mathew, <, mathew, @, mantis.co.uk, ..."
5,From: uabdpo.dpo.uab.edu!gila005 (Stephen Holl...,2.0,sci.med,"[From, :, uabdpo.dpo.uab.edu, !, gila005, (, S..."
6,From: ron.roth@rose.com (ron roth) Subject: Se...,2.0,sci.med,"[From, :, ron.roth, @, rose.com, (, ron, roth,..."
7,From: bates@spica.ucsb.edu (Andrew M. Bates) S...,1.0,comp.graphics,"[From, :, bates, @, spica.ucsb.edu, (, Andrew,..."
8,From: billj@b11.b11.ingr.com (Bill Jones) Subj...,1.0,comp.graphics,"[From, :, billj, @, b11.b11.ingr.com, (, Bill,..."
9,From: madhaus@netcom.com (Maddi Hausmann) Subj...,0.0,alt.atheism,"[From, :, madhaus, @, netcom.com, (, Maddi, Ha..."


In [59]:
count_vect.transform(['Something completely new.']).toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [60]:
count_vect.transform(['\xed\xe5littin Something completely new \xed\xe5littin.']).toarray()

array([[0, 0, 0, ..., 0, 0, 2]])

In [61]:
plot_x = ["term_"+str(i) for i in count_vect.get_feature_names()[0:20]]

In [62]:
plot_x

['term_00',
 'term_000',
 'term_0000',
 'term_0000001200',
 'term_000005102000',
 'term_0001',
 'term_000100255pixel',
 'term_00014',
 'term_000406',
 'term_0007',
 'term_000usd',
 'term_0010',
 'term_001004',
 'term_0010580b',
 'term_001125',
 'term_001200201pixel',
 'term_0014',
 'term_001642',
 'term_00196',
 'term_002']

In [63]:
plot_y = ["doc_"+ str(i) for i in list(X.index)[0:20]]

In [64]:
plot_y

['doc_0',
 'doc_1',
 'doc_2',
 'doc_3',
 'doc_4',
 'doc_5',
 'doc_6',
 'doc_7',
 'doc_8',
 'doc_9',
 'doc_10',
 'doc_11',
 'doc_12',
 'doc_13',
 'doc_14',
 'doc_15',
 'doc_16',
 'doc_17',
 'doc_18',
 'doc_19']

In [65]:
plot_z = X_counts[0:20, 0:20].toarray()

In [66]:
# to plot
py.iplot(ta.plot_heat_map(plot_x, plot_y, plot_z))

#### Dimensionality Reduction using PCA

In [67]:
from sklearn.decomposition import PCA


In [68]:
print X_counts.shape

(2257, 35788)


In [69]:
X_reduced = PCA(n_components=3).fit_transform(X_counts.toarray())

In [70]:
X_reduced.shape

(2257, 3)

In [71]:
trace1 = ta.get_trace(X_reduced, X["category_name"], "alt.atheism", "rgb(71,233,163)")
trace2 = ta.get_trace(X_reduced, X["category_name"], "soc.religion.christian", "rgb(52,133,252)")
trace3 = ta.get_trace(X_reduced, X["category_name"], "comp.graphics", "rgb(229,65,136)")
trace4 = ta.get_trace(X_reduced, X["category_name"], "sci.med", "rgb(99,151,68)")

In [74]:
data = [trace1, trace2, trace3, trace4]

In [83]:
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

#### Discretization and Binarization

In [77]:
from sklearn import preprocessing, metrics, decomposition, pipeline, dummy

In [78]:
mlb = preprocessing.LabelBinarizer()

In [79]:
mlb.fit(X.category_name)

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [80]:
mlb.classes_

array(['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian'], 
      dtype='|S22')

In [81]:
X['bin_category'] = mlb.transform(X['category_name']).tolist()

In [82]:
X[0:20]

,text,category,category_name,unigrams,bin_category
0,From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul C...,2.0,sci.med,"[From, :, dpc47852, @, uxa.cso.uiuc.edu, (, Da...","[0, 0, 1, 0]"
1,From: yoo@engr.ucf.edu (Hoi Yoo) Subject: look...,1.0,comp.graphics,"[From, :, yoo, @, engr.ucf.edu, (, Hoi, Yoo, )...","[0, 1, 0, 0]"
2,From: fernandeza@merrimack.edu Subject: Re: Th...,3.0,soc.religion.christian,"[From, :, fernandeza, @, merrimack.edu, Subjec...","[0, 0, 0, 1]"
3,From: mcelwre@cnsvax.uwec.edu Subject: NATURAL...,2.0,sci.med,"[From, :, mcelwre, @, cnsvax.uwec.edu, Subject...","[0, 0, 1, 0]"
4,From: mathew <mathew@mantis.co.uk> Subject: Re...,0.0,alt.atheism,"[From, :, mathew, <, mathew, @, mantis.co.uk, ...","[1, 0, 0, 0]"
5,From: uabdpo.dpo.uab.edu!gila005 (Stephen Holl...,2.0,sci.med,"[From, :, uabdpo.dpo.uab.edu, !, gila005, (, S...","[0, 0, 1, 0]"
6,From: ron.roth@rose.com (ron roth) Subject: Se...,2.0,sci.med,"[From, :, ron.roth, @, rose.com, (, ron, roth,...","[0, 0, 1, 0]"
7,From: bates@spica.ucsb.edu (Andrew M. Bates) S...,1.0,comp.graphics,"[From, :, bates, @, spica.ucsb.edu, (, Andrew,...","[0, 1, 0, 0]"
8,From: billj@b11.b11.ingr.com (Bill Jones) Subj...,1.0,comp.graphics,"[From, :, billj, @, b11.b11.ingr.com, (, Bill,...","[0, 1, 0, 0]"
9,From: madhaus@netcom.com (Maddi Hausmann) Subj...,0.0,alt.atheism,"[From, :, madhaus, @, netcom.com, (, Maddi, Ha...","[1, 0, 0, 0]"
